##### 版权所有 2024 Google LLC.

In [ ]:
# @title 根据 Apache 许可证 2.0 版本（“许可证”）授权；
# 除非遵循许可证，否则不得使用本文件。
# 您可以通过以下地址获取许可证副本：
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# 除非适用法律要求或书面同意，
# 否则根据“现状”分发本软件，不附带任何明示或暗示的保证。
# 详见许可证中关于权限与限制的具体条款。

# CodeGemma - 常见用例
本笔记本通过合适的提示（prompting）展示 Gemma 能够解决的基础任务。
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/CodeGemma/[CodeGemma_1]Common_use_cases.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
</table>

In [ ]:
import os

# 我这里使用的配置文件直接导入为环境变量
os.environ["KAGGLE_USERNAME"] = "你的kaggle用户名"
os.environ["KAGGLE_KEY"] = "你的kaggle key"

### 安装依赖
运行下方单元格，安装所有必需的依赖包。

In [ ]:
%pip install -q -U keras keras-nlp

## 探索提示能力

### CodeGemma

CodeGemma 模型是“文本到文本”以及“文本到代码”的纯解码器模型，专门用于代码补全与代码生成任务。CodeGemma 2B 与 7B 版本特别针对**代码填充（infilling）**场景进行了调优。

本示例利用 CodeGemma 的 **FIM（Fill-in-the-middle）** 能力，根据上下文自动补全代码。这在代码编辑器中尤为实用：当光标位于某段代码中间时，模型可根据前后文自动插入缺失内容。

CodeGemma 提供 4 个用户自定义标记：
- `<|fim_prefix|>`
- `<|fim_suffix|>`
- `<|fim_middle|>`
- `<|file_separator|>`（用于多文件上下文）

接下来我们将用这些标记定义常量，并在后续单元格中演示具体用法。

In [ ]:
import os
import keras
import keras_nlp
from google.colab import userdata

keras.config.set_floatx("bfloat16")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
# 加载 CodeGemma
codegemma = keras_nlp.models.GemmaCausalLM.from_preset("code_gemma_1.1_2b_en")

In [ ]:
# 定义标记常量
BEFORE_CURSOR = "<|fim_prefix|>"
AFTER_CURSOR = "<|fim_suffix|>"
AT_CURSOR = "<|fim_middle|>"
FILE_SEPARATOR = "<|file_separator|>"
END_TOKEN = codegemma.preprocessor.tokenizer.end_token
stop_tokens = (BEFORE_CURSOR, AFTER_CURSOR, AT_CURSOR, FILE_SEPARATOR, END_TOKEN)

In [ ]:
stop_token_ids = tuple(
    codegemma.preprocessor.tokenizer.token_to_id(x) for x in stop_tokens
)

#### 提示示例：代码填充

In [ ]:
import re


# 辅助函数
def split_response_by_token(response):
    mapping = {}
    parts = re.split(r"(<\|[^\|\>]+\|\>)", response)
    parts = [item for item in parts if len(item)]
    for token in stop_tokens[:3]:
        mapping[token] = ""

        try:
            idx = parts.index(token)
            if parts[idx + 1] not in stop_tokens:
                mapping[token] = parts[idx + 1]
        except (ValueError, IndexError):
            pass

    return mapping

In [ ]:
prefix = "def calculate_area_of_rectangle(a: int, b: int) -> int:"
suffix = "\n    return area"
prompt = f"<|fim_prefix|>{prefix}<|fim_suffix|>{suffix}<|fim_middle|>"

response = codegemma.generate(prompt, stop_token_ids=stop_token_ids)
parts = split_response_by_token(response)

print("--- 原始响应 ---")
print(response)

print("\n--- 生成的（FIM）代码片段： ---")
print(parts[AT_CURSOR])

print("\n--- 完整函数： ---")
print(parts[BEFORE_CURSOR], parts[AT_CURSOR], parts[AFTER_CURSOR])

#### 提示示例：代码生成
_注意：虽然 2B 版 CodeGemma 主要面向代码补全场景，但它也能完成基础代码生成任务。如需更佳的代码生成效果，建议使用经过指令调优的 7B 模型。_

In [ ]:
prompt = """用一行 Python 代码判断某年是否为闰年。
示例：
>>> is_a_leap_year(2016)
True
>>> is_a_leap_year(2001)
False
>>> is_a_leap_year(2052)
True
def is_a_leap_year(year: int) -> bool:"""

response = codegemma.generate(prompt, max_length=128)
print(response)

In [ ]:
# 需要重启会话以释放 GPU 内存，并加载新模型
get_ipython().kernel.do_shutdown(True)